In [ ]:
from z3 import *
import numpy as np
import random
import arviz as az

import os, sys
sys.path.append(os.path.join("../../../"))

# our library
from src.mcmc_sat import smt,mcmc

## Model definition (in Z3)

We start by defining a model as an SMT problem using Z3

In [ ]:
# We create five Integer variables
num_vars = 3
x = [Int(f'x{i}') for i in range(num_vars)]

In [ ]:
# We define the model using the variables above

s = Solver() # get an instance of a Z3 solver

# model constraints
s.add(x[0]>=0)
s.add(x[1]>=0)
s.add(x[2]>=0)
s.add(x[0]<=3)
s.add(x[1]<=3)
s.add(x[2]<=6)
s.add(x[0] + x[1] == x[2])

## Storing the model

We store the Z3 problem in a smt2 format. This is the input format required by Megasampler.

In [ ]:
MEGASAMPLER_INPUT_DIR = 'megasampler_input'
megasampler_input_file = 'z3_synth_problem_2.smt2'
megasampler_input_filepath = f'{MEGASAMPLER_INPUT_DIR}/{megasampler_input_file}'

smt.save_smt2(solver=s, filepath=megasampler_input_filepath)

## Executing Megasampler

Now we execute megasampler to produce 10000 samples (by default). The input to the function is the file where the smt2 version of the Z3 problem is stored.

In [ ]:
# We execute megasampler \o/
MEGASAMPLER_OUTPUT_DIR = 'megasampler_output'
smt.execute_megasampler(input_filepath=megasampler_input_filepath, algo = 'SMT',
                        output_dir=MEGASAMPLER_OUTPUT_DIR)

Megasampler stores the samples in a file. The file is stored in the directory passed as parameter in the above function. In what follows, we parse this file to produce a list of samples. Each sample is encoded as a dictionary where the key is the variable name and the value the sample.

In [ ]:
# Define output files with samples
file_samples = f'{MEGASAMPLER_OUTPUT_DIR}/{megasampler_input_file}.samples'

In [ ]:
# We parse the samples
samples = smt.parse_megasamples(file_samples)

## We run our MCMC algo

Finally, we run our MCMC algorithm. It takes as input the parsed samples. Currently, we always accept the samples. There is no likelihood or prior.

In [ ]:
# We run MCMC using the "megasamples"
trace = mcmc.sample_mh_trace(10000,4,samples)

In [ ]:
trace

## Posterior

We conclude by showing the posterior trace plots and summary statistics for each variable.

In [ ]:
# Plot the posterior
import matplotlib.pyplot as plt
az.plot_trace(trace);
plt.savefig('megasampler_trace_plot.pdf');

In [ ]:
# Diagnostics
az.summary(trace)